In [1]:
import random, string, requests, time, json
from tqdm import tqdm
from threading import Thread

localhost = "http://127.0.0.1"
url = localhost + ":5000/"


In [2]:
records = [
    {
        "Stud_id" : i,
        "Stud_name" : ''.join(random.choices(string.ascii_uppercase, k = 6)),
        "Stud_marks" : random.randint(0, 100)
    }
    
    for i in range(0, 10000)
]

In [3]:
def read_target(batch_size : int = 10):
    lo = random.randint(0, 10000)
    hi = lo + batch_size
    r = requests.post(url + "read", data = json.dumps({"Stud_id" : {"low" : lo, "high" : hi}}))

def readN(N : int = 1000, batch_size : int = 10):
    
    start = time.time()
    
    
    threads = []
    for i in tqdm(range(N)):
        threads.append(Thread(target = lambda: read_target(batch_size,)))
        threads[-1].start()
    for i in tqdm(range(N)):
        threads[i].join()
        

    end = time.time()
    
    return f"{N} read requests with batches of {batch_size} took {end-start:.2f} seconds."

In [4]:
def write_target(batch_size : int = 10):
    data = random.choices(records, k = batch_size)
    r = requests.post(url + "write", data = json.dumps({"data" : data}))

def writeN(N : int = 1000, batch_size : int = 10):
    
    start = time.time()
    
    threads = []
    for i in tqdm(range(N)):
        threads.append(Thread(target = lambda: write_target(batch_size,)))
        threads[-1].start()
    for i in tqdm(range(N)):
        threads[i].join()
    
    end = time.time()
    
    return f"{N} write requests with batches of {batch_size} took {end-start:.2f} seconds."

# A-1

Report the read and write speed for 10000 writes and 10000 reads in the default configuration given

In [8]:
# Initialise

init = {
    "N":3,
    "schema":{"columns":["Stud_id","Stud_name","Stud_marks"],"dtypes":["Number","String","String"]},
    "shards":[{"Stud_id_low":0, "Shard_id": "sh1", "Shard_size":4096},
            {"Stud_id_low":4096, "Shard_id": "sh2", "Shard_size":4096},
            {"Stud_id_low":8192, "Shard_id": "sh3", "Shard_size":4096}],
    "servers":{"Server0":["sh1","sh2"],
            "Server1":["sh2","sh3"],
            "Server2":["sh1","sh3"]}
}

r = requests.post("http://localhost:5000/init", json=init)

print(r.text)

{"Added Servers":["Server_1","Server_2","Server_3"],"message":"Configured Database","status":"success"}



In [10]:
# Check Status

r = requests.get("http://localhost:5000/status")
r.json()

{'N': 3,
 'schema': {'columns': ['Stud_id', 'Stud_name', 'Stud_marks'],
  'dtypes': ['Number', 'String', 'String']},
 'servers': {'Server0': ['sh1', 'sh2'],
  'Server1': ['sh2', 'sh3'],
  'Server2': ['sh1', 'sh3']},
 'shards': [{'Primary_server': 'Server0',
   'Shard_id': 'sh1',
   'Shard_size': 4096,
   'Stud_id_low': 0},
  {'Primary_server': 'Server0',
   'Shard_id': 'sh2',
   'Shard_size': 4096,
   'Stud_id_low': 4096},
  {'Primary_server': 'Server1',
   'Shard_id': 'sh3',
   'Shard_size': 4096,
   'Stud_id_low': 8192}]}

In [11]:
## Write


writeN()

100%|██████████| 1000/1000 [00:00<00:00, 69720.31it/s]


'1000 write requests with batches of 10 took 1.24 seconds.'

In [12]:
# Read

readN()

100%|██████████| 1000/1000 [00:00<00:00, 38969.29it/s]


'1000 read requests with batches of 10 took 1.16 seconds.'

# A-2

Increase the number of shard replicas (to 7) from the configuration (init endpoint). Report the write speed down for
10000 writes and read speed up for 10000 reads.

In [15]:
# Initialise

init = {
    "N":7,
    "schema":{"columns":["Stud_id","Stud_name","Stud_marks"],"dtypes":["Number","String","String"]},
    "shards":[{"Stud_id_low":0, "Shard_id": "sh1", "Shard_size":4096},
            {"Stud_id_low":4096, "Shard_id": "sh2", "Shard_size":4096},
            {"Stud_id_low":8192, "Shard_id": "sh3", "Shard_size":4096}],
    "servers":{
            "Server0":["sh1", "sh2", "sh3"],
            "Server1":["sh1", "sh2", "sh3"],
            "Server2":["sh1", "sh2", "sh3"],
            "Server3":["sh1", "sh2", "sh3"],
            "Server4":["sh1", "sh2", "sh3"],
            "Server5":["sh1", "sh2", "sh3"],
            "Server6":["sh1", "sh2", "sh3"],
            }
}

r = requests.post("http://localhost:5000/init", json=init)

print(r.text)

{"Added Servers":["Server_1","Server_2","Server_3","Server_4","Server_5","Server_6","Server_7"],"message":"Configured Database","status":"success"}



In [16]:
# Check Status

r = requests.get(f"{localhost}:5000/status")
r.json()

{'N': 7,
 'schema': {'columns': ['Stud_id', 'Stud_name', 'Stud_marks'],
  'dtypes': ['Number', 'String', 'String']},
 'servers': {'Server0': ['sh1', 'sh2', 'sh3'],
  'Server1': ['sh1', 'sh2', 'sh3'],
  'Server2': ['sh1', 'sh2', 'sh3'],
  'Server3': ['sh1', 'sh2', 'sh3'],
  'Server4': ['sh1', 'sh2', 'sh3'],
  'Server5': ['sh1', 'sh2', 'sh3'],
  'Server6': ['sh1', 'sh2', 'sh3']},
 'shards': [{'Primary_server': 'Server0',
   'Shard_id': 'sh1',
   'Shard_size': 4096,
   'Stud_id_low': 0},
  {'Primary_server': 'Server0',
   'Shard_id': 'sh2',
   'Shard_size': 4096,
   'Stud_id_low': 4096},
  {'Primary_server': 'Server0',
   'Shard_id': 'sh3',
   'Shard_size': 4096,
   'Stud_id_low': 8192}]}

In [17]:
writeN()

100%|██████████| 1000/1000 [00:00<00:00, 487823.21it/s]


'1000 write requests with batches of 10 took 1.19 seconds.'

In [18]:
readN()

100%|██████████| 1000/1000 [00:00<00:00, 134553.57it/s]


'1000 read requests with batches of 10 took 1.08 seconds.'

# A-3

Increase the number of Servers (to 10) by adding new servers and increase the number of shards (shard to 6, shard
replicas to 8). Define the (init endpoint) configurations according to your choice. Report the write speed up for 10000
writes and read speed up for 10000 reads

In [19]:
# Initialise

init = {
    "N":10,
    "schema":{"columns":["Stud_id","Stud_name","Stud_marks"],"dtypes":["Number","String","String"]},
    "shards":[{"Stud_id_low":0, "Shard_id": "sh1", "Shard_size":4096},
            {"Stud_id_low":4096, "Shard_id": "sh2", "Shard_size":4096},
            {"Stud_id_low":8192, "Shard_id": "sh3", "Shard_size":4096},
            {"Stud_id_low":12288, "Shard_id": "sh4", "Shard_size":4096},
            {"Stud_id_low":16384, "Shard_id": "sh5", "Shard_size":4096},
            {"Stud_id_low":20480, "Shard_id": "sh6", "Shard_size":4096}],
    "servers":{
            "Server0":["sh1", "sh2", "sh3", "sh4", "sh6"],
            "Server1":["sh1", "sh2", "sh3", "sh4", "sh6"],
            "Server2":["sh1", "sh2", "sh3", "sh5", "sh6"],
            "Server3":["sh1", "sh2", "sh3", "sh5", "sh6"],
            "Server4":["sh1", "sh2", "sh4", "sh5", "sh6"],
            "Server5":["sh1", "sh2", "sh4", "sh5", "sh6"],
            "Server6":["sh1", "sh3", "sh4", "sh5", "sh6"],
            "Server7":["sh1", "sh3", "sh4", "sh5", "sh6"],
            "Server8":["sh2", "sh3", "sh4", "sh5"],
            "Server9":["sh2", "sh3", "sh4", "sh5"]
            }
}

r = requests.post("http://localhost:5000/init", json=init)

print(r.text)

{"Added Servers":["Server_1","Server_2","Server_3","Server_4","Server_5","Server_6","Server_7","Server_8","Server_9","Server_10"],"message":"Configured Database","status":"success"}



In [20]:
# Check Status

r = requests.get(f"{localhost}:5000/status")
r.json()

{'N': 10,
 'schema': {'columns': ['Stud_id', 'Stud_name', 'Stud_marks'],
  'dtypes': ['Number', 'String', 'String']},
 'servers': {'Server0': ['sh1', 'sh2', 'sh3', 'sh4', 'sh6'],
  'Server1': ['sh1', 'sh2', 'sh3', 'sh4', 'sh6'],
  'Server2': ['sh1', 'sh2', 'sh3', 'sh5', 'sh6'],
  'Server3': ['sh1', 'sh2', 'sh3', 'sh5', 'sh6'],
  'Server4': ['sh1', 'sh2', 'sh4', 'sh5', 'sh6'],
  'Server5': ['sh1', 'sh2', 'sh4', 'sh5', 'sh6'],
  'Server6': ['sh1', 'sh3', 'sh4', 'sh5', 'sh6'],
  'Server7': ['sh1', 'sh3', 'sh4', 'sh5', 'sh6'],
  'Server8': ['sh2', 'sh3', 'sh4', 'sh5'],
  'Server9': ['sh2', 'sh3', 'sh4', 'sh5']},
 'shards': [{'Primary_server': 'Server0',
   'Shard_id': 'sh1',
   'Shard_size': 4096,
   'Stud_id_low': 0},
  {'Primary_server': 'Server0',
   'Shard_id': 'sh2',
   'Shard_size': 4096,
   'Stud_id_low': 4096},
  {'Primary_server': 'Server0',
   'Shard_id': 'sh3',
   'Shard_size': 4096,
   'Stud_id_low': 8192},
  {'Primary_server': 'Server0',
   'Shard_id': 'sh4',
   'Shard_size': 

In [21]:
writeN()

100%|██████████| 1000/1000 [00:00<00:00, 512000.00it/s]


'1000 write requests with batches of 10 took 1.17 seconds.'

In [22]:
readN()

100%|██████████| 1000/1000 [00:00<00:00, 348798.67it/s]


'1000 read requests with batches of 10 took 1.20 seconds.'

# A-4 

Finally, check all the endpoints and ensure their correctness. Manually drop a server container and show that the load
balancer spawns a new container and copies the shard entries from other replicas.